# Import packages and data

In [1]:
from datetime import date
import glob
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime, timedelta
import time
import pytz
import os

In [2]:
# Data location 
folder_external = r'C:\Users\BI-LOCAL\Documents\Python Scripts\Travix\data\external\\'

In [3]:
# Network locations for data import & export### Import data from BI drive
input_TravixDatadump = r'\\192.168.140.37\\Business-Share\\BI resources\\Dashboards\\Travix\\# Data\\Travix Datadump\\'
export_BI_drive= r'\\192.168.140.37\\Business-Share\\BI resources\\Dashboards\\Travix\\# Travix NL\\# Data\\'

### Import data from BI drive

In [4]:
#Specify all bi-report files directory 
All_TR_Data = glob.glob(input_TravixDatadump + "/*.csv")
#Specify files you want to exclude
Data_TR_2018= glob.glob(input_TravixDatadump + "/2018*.csv")
#Data_TR_201901= glob.glob(input_TravixDatadump + "/201901*.csv")
#Data_TR_201904= glob.glob(input_TravixDatadump + "/201904*.csv")
# Specify relevant di-report files 
Select_TR_Data = set(All_TR_Data)-set(Data_TR_2018)

In [5]:
#Import historical data
Travix_base_df = pd.concat((pd.read_csv(f,low_memory=False) for f in Select_TR_Data),sort=True)

### Import downloaded SFTP Archive data 

In [6]:
# Import all distributor data from local folder 
AllData = glob.glob(folder_external + "/external_travix*.csv")
Travix_SFTP_df = pd.concat((pd.read_csv(f, low_memory=False) for f in AllData))

In [7]:
# extract the date & time from string in LastUpdatedDate column and convert type of column to datetime64
Travix_SFTP_df['LastUpdatedDate'] = pd.to_datetime(Travix_SFTP_df['LastUpdatedDate'].astype(str).str[:19],format='%Y-%m-%d %H:%M:%S')

In [8]:
# Select the latest session ID based on last updated date column and drop the rest
Travix_SFTP_df = Travix_SFTP_df.sort_values('LastUpdatedDate').drop_duplicates(['SessionId', 'OrderNumber'], keep='last')

### Merge Travix data files

In [9]:
# Merge Travix csv files in folders with other csv files not in folders
travix_df = pd.concat([Travix_base_df,Travix_SFTP_df], sort = False)

### Travix data cleaning

# Data Cleaning

In [10]:
### Travix data cleaning# Select data range starting from the time we went live
Start_date = ('2018-05-29 00:00:00')
Start_date = pd.to_datetime(Start_date)
Start_date

Timestamp('2018-05-29 00:00:00')

In [11]:
#Excluding time from the Order Date and changing orderdate to date format
travix_df['OrderDateUTC'] = pd.to_datetime(travix_df['OrderDateUTC'],infer_datetime_format=True,exact=False).dt.date
travix_df['OrderDateUTC'] = pd.to_datetime(travix_df['OrderDateUTC'])

In [12]:
travix_base_df=travix_df.copy()

## Removing incomplete last day

In [13]:
# start date report
End_date = travix_base_df['OrderDateUTC'].max()
End_date

Timestamp('2020-05-12 00:00:00')

In [14]:
# Remove incomplete last day in a dataset
travix_base_df=travix_base_df[travix_base_df['OrderDateUTC']< End_date]

In [15]:
#Check maximum date
travix_base_df['OrderDateUTC'].max()

Timestamp('2020-05-11 00:00:00')

## Selecting NL brand only

In [16]:
#Limit data to NL brand only
travix_base_df=travix_base_df[travix_base_df['Brand'].isin(['CHEAPTICKETSNL', 'BUDGETAIR', 'VLIEGWINKEL'])]
#check brands in the data
travix_base_df['Brand'].value_counts()

CHEAPTICKETSNL    420942
BUDGETAIR         144395
VLIEGWINKEL        71996
Name: Brand, dtype: int64

In [17]:
#rename Brand to Brand/Channel
travix_base_df=travix_base_df.rename(columns={'Brand':'Brand/Channel'})

## Haul

In [18]:
#Check haul column before adding 'No insurance' category
travix_base_df['Haul'].value_counts()

Short Haul    404294
Long Haul     209027
Name: Haul, dtype: int64

In [19]:
# Creating a new column that also indicates 'No haul' if haul type is not indicated in the Haul column
#fill missing values with 'No insurance'
travix_base_df.Haul.fillna(value='No Hual', inplace=True)

In [20]:
#Check haul column after adding 'No insurance' category
travix_base_df['Haul'].value_counts()

Short Haul    404294
Long Haul     209027
No Hual        24012
Name: Haul, dtype: int64

## Policy Type

In [21]:
#Creating a new column that also indicates 'No insurance' if a customer didnt buy insurance
#making a copy of Policy type
travix_base_df['Policytype']=travix_base_df['PolicyType']
#fill missing values with 'No insurance'
travix_base_df.Policytype.fillna(value='No insurance', inplace=True)

In [22]:
#rename policy type categories to travel, cancellation and combi insurance
travix_base_df['Policytype'].replace(to_replace=('HEPSTAR-CANCELLATION','HEPSTAR-COMBINED-INSURANCE','HEPSTAR-TRAVEL', 'HEPSTAR-TRAVELCANCELLATION', 'reissverzekering', 'Travel Insurance ', 'cancellationinsurance'), value=('Cancellation Insurance','Combi Insurance','Travel Insurance', 'Combi Insurance', 'Travel Insurance', 'Travel Insurance', 'Cancellation Insurance' ), inplace=True)
#Check
travix_base_df['Policytype'].value_counts()

No insurance              587130
CANCELLATION-INSURANCE     22633
Cancellation Insurance     13113
COMBINED-INSURANCE          4242
TRAVEL-INSURANCE            3570
Combi Insurance             2744
Travel Insurance            2515
EXTRAS                      1386
Name: Policytype, dtype: int64

In [23]:
travix_base_df['Policy_type'] =''
travix_base_df['Policy_type'] = np.where(travix_base_df['Policytype'] == 'CANCELLATION-INSURANCE', 'Cancellation Insurance', travix_base_df['Policy_type'])
travix_base_df['Policy_type'] = np.where(travix_base_df['Policytype'] == 'COMBINED-INSURANCE', 'Travel+Cancellation Insurance', travix_base_df['Policy_type'])
travix_base_df['Policy_type'] = np.where(travix_base_df['Policytype'] == 'TRAVEL-INSURANCE', 'Travel Insurance', travix_base_df['Policy_type'])
travix_base_df['Policy_type'] = np.where(travix_base_df['Policytype'] == 'Combi Insurance', 'Travel+Cancellation Insurance', travix_base_df['Policy_type'])
travix_base_df['Policy_type'] = np.where(travix_base_df['Policy_type'] == '',travix_base_df['Policytype'], travix_base_df['Policy_type'])
travix_base_df['Policy_type'].value_counts()

No insurance                     587130
Cancellation Insurance            35746
Travel+Cancellation Insurance      6986
Travel Insurance                   6085
EXTRAS                             1386
Name: Policy_type, dtype: int64

In [24]:
# Split rows with Travel Insurance+Cancellation Insurance into two different rows
travix_base_df = \
(travix_base_df.set_index(travix_base_df.columns.drop('Policy_type',1).tolist())
   .Policy_type.str.split('+', expand=True)
   .stack()
   .reset_index()
   .rename(columns={0:'Policy_type'})
   .loc[:, travix_base_df.columns]
)

In [25]:
travix_base_df['Policy_type'].value_counts()

No insurance              587130
Cancellation Insurance     42732
Travel                      6986
Travel Insurance            6085
EXTRAS                      1386
Name: Policy_type, dtype: int64

## Travix booking

In [26]:
# Creating a new column that indicates 1 if a customer bought flight ticket 
travix_base_df['Travix_booking'] = travix_base_df['CustomerIdentifier'].where (travix_base_df['CustomerIdentifier'].isnull(), 1).fillna(0).astype(int)
#check if the new column is correct
travix_base_df['CustomerIdentifier'].count() - travix_base_df['Travix_booking'].sum()

0

In [27]:
#check if every row has brand identified
travix_base_df['Brand/Channel'].count()

644319

## Insurance Purchase

In [28]:
# Creating a new column that indicates 1 if a customer bought insurance and 0 otherwise
#setting insurance purchase to either 1 or zero
travix_base_df['Insurance_purch'] = travix_base_df['PolicyType'].where (travix_base_df['PolicyType'].isnull(), 1).fillna(0).astype(int)
#check if the new column is correct
travix_base_df['PolicyType'].count() - travix_base_df['Insurance_purch'].sum()

0

In [29]:
#Check
travix_base_df['Insured'].value_counts()-travix_base_df['Insurance_purch'].sum()

False    529936
True          0
false    -57185
fa       -57188
Name: Insured, dtype: int64

## Test product

In [30]:
#Remove test products 
travix_base_df.drop(travix_base_df[travix_base_df['Policy_type']=='test'].index, inplace=True)

## Year, Month Week variables

In [31]:
# create order month/year variable
travix_base_df['Order_month'] = travix_base_df['OrderDateUTC'].dt.month
travix_base_df['Order_year'] = travix_base_df['OrderDateUTC'].dt.year
# create  order week variable
travix_base_df['Orderweek'] = travix_base_df['OrderDateUTC'].dt.week
# create order month and order year variable
travix_base_df['YearMonth'] = travix_base_df['OrderDateUTC'].apply(lambda x:x.strftime('%Y-%m'))
#create year and week variable
travix_base_df['Yearweek'] = travix_base_df['OrderDateUTC'].apply(lambda x:x.strftime('%Y-%W'))

In [32]:
#convert order_year to string (so as to be able to join a multiindex level when creating a pivot table)
travix_base_df['Order_year']=travix_base_df['Order_year'].apply(str)

## Duplicates

In [33]:
#select data from the day we went live
Tr_live_df = travix_base_df[travix_base_df['OrderDateUTC']>='2018-05-29 00:00:00']
Tr_live_df=Tr_live_df.copy()

In [34]:
Tr_live_df.shape

(644319, 65)

In [35]:
#select data before we went live
Tr_B4_live= travix_base_df[travix_base_df['OrderDateUTC']<'2018-05-29 00:00:00']
Tr_B4_live=Tr_B4_live.copy()

In [36]:
#drop duplicates with the same ordernumber and policy type from the beggining of phase 1
Tr_live_df.drop_duplicates(['OrderNumber', 'PolicyType'],keep='first',inplace=True)

In [37]:
# create a list of columns which are constant per order number
constant_columns = ['OrderNumber', 'OrderDateUTC', 'Brand/Channel', 'OneWayOrReturn', 'ReceivedDiscount','DepartureDate', 'ReturnDate', 'Duration',
                    'Insured', 'Haul','SessionId', 'ChannelType', 'CustomerIdentifier', 'TotalNumberOfOrders', 'DomesticOrInternational',
                     'Airline','NumberOfPassengers','NumberOfInfants','NumberOfChildren','NumberOfAdults', 'BookerDateOfBirth','OriginAirportCode',
                   'FlewEconomyClass','FlewFirstClass','FlewBusinessClass','FlewPremiumEconomyClass','Order_year', 'Orderweek','Order_month', 
                    'PolicyNumber', 'Travix_booking', 'Insurance_purch','InsuranceSalesCurrency', 'OrderCanceled', 'Policy_type', 'YearMonth', 'Yearweek',
                   'BookerCountry_Code','OriginCountryCode', 'DestinationAirportCode', 'DestinationCountryCode',
                    'PolicyName', 'InsuranceStartDate', 'InsuranceEndDate', 'Supplier','PolicyNumber']                            

In [38]:
#join the dataset with the constant columns
excl_dups_df =Tr_live_df[constant_columns]
# create index based on the 'OrderNumber' column
excl_dups_df.set_index('OrderNumber', inplace=True)

In [39]:
#create a new database with a list of names for columns to be summed per order number
columns_to_sum = ['InsuranceCost_AdministrativeCost',
                  'InsuranceCost_Premium', 'InsuranceCost_InsuranceTax',
                  'InsuranceCost_AdditionalCoverageCost', 'TotalInsuranceCost',
                  'InsuredAmount','TotalTicketSale_EUR']                             

In [40]:
# create a new dataframe with sums for each order number
sums_excl_df =Tr_live_df.groupby('OrderNumber')[columns_to_sum].sum()

In [41]:
Tr_live_df['PolicyType'].value_counts()

CANCELLATION-INSURANCE        12383
HEPSTAR-CANCELLATION           9699
COMBINED-INSURANCE             2327
HEPSTAR-TRAVELCANCELLATION     1980
TRAVEL-INSURANCE               1909
HEPSTAR-TRAVEL                 1836
EXTRAS                          799
Cancellation Insurance            4
Travel Insurance                  1
Name: PolicyType, dtype: int64

In [42]:
 #MERGE CONSTANT AND TRANSFORMED DATASETS
# merge the base data frame with the one containing sums
excl_dups_df = excl_dups_df.merge(sums_excl_df, left_index=True, right_index=True)

In [43]:
#Reset index so as to merge the two different datasets
excl_dups_df.reset_index(inplace=True)

In [44]:
#drop duplicates by ordernumber
excl_dups_df.drop_duplicates(['OrderNumber'],keep='first',inplace=True)

In [45]:
#output_df[output_df['Policy_names']=='HEPSTAR-TRAVEL + HEPSTAR-TRAVELCANCELLATION']['OrderNumber']

In [46]:
#Merge the two dataset (before and after we went live)
Travix_data = pd.concat([excl_dups_df,Tr_B4_live], join_axes = [excl_dups_df.columns])

C:\Users\BI-LOCAL\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  


In [47]:
Travix_data.shape

(502579, 53)

## Travix revenue

In [48]:
#Calculate 58% Insurance premium
Travix_data['58%Insurance_Premium']=0.58*Travix_data['InsuranceCost_Premium']

In [49]:
#check
test= Travix_data[['58%Insurance_Premium', 'InsuranceCost_Premium']]
test[test.InsuranceCost_Premium.notnull()].head(5)

,58%Insurance_Premium,InsuranceCost_Premium
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [50]:
#function to add a columns with masterpolicyfee
Travix_data['Masterpolicy_fee'] = Travix_data['Policy_type'].where (Travix_data['Policy_type'].isnull(), 6).fillna(0).astype(int)
Travix_data['Masterpolicy_fee'].value_counts()

6    502579
Name: Masterpolicy_fee, dtype: int64

In [51]:
#function to add a columns indicating the phase in which the orders occured
def Travix_Phase(Travix_data):
    if Travix_data['OrderDateUTC'] >= Start_date:
        return 'Phase1&2'
    else:
        return 'Phase0'
Travix_data['Phase'] = Travix_data.apply (lambda row: Travix_Phase (row),axis=1)

In [52]:
#function to add a columns with masterpolicyfee without tax (at the beginning of Phase 2a going onwards)
def MP_fee_exc_tax(Travix_data):
    if (Travix_data['Insurance_purch'] == 1) and (Travix_data['OrderDateUTC']>= Start_date):
        return 5.8
    if (Travix_data['Insurance_purch'] == 1) and (Travix_data['OrderDateUTC']< Start_date):
        return 6
    else:
        return 0
Travix_data['MP_fee_exc_tax&HS'] = Travix_data.apply (lambda row: MP_fee_exc_tax (row),axis=1)

In [53]:
Travix_data['MP_fee_exc_tax&HS'].value_counts()

0.0    471642
5.8     30937
Name: MP_fee_exc_tax&HS, dtype: int64

In [54]:
Travix_data['Insurance_purch'].value_counts()

0    471642
1     30937
Name: Insurance_purch, dtype: int64

In [55]:
# Travix Revenue
Travix_data['Travix_revenue']= Travix_data['MP_fee_exc_tax&HS']+Travix_data['58%Insurance_Premium']

In [56]:
#create a column indicating the total retail value
Travix_data['TotalRetail_value']=Travix_data['TotalInsuranceCost']

## Cancelled Orders

In [57]:
Travix_data['OrderCanceled'].value_counts()

False    485720
True      16858
Name: OrderCanceled, dtype: int64

In [58]:
# Removing Cancelled Orders starting from phase 1 going onwards
#function to add a columns that indicates if a policy was cancelled or not
def Order_cancelled(Travix_data):
    if (Travix_data['OrderCanceled'] == True) and (Travix_data['OrderDateUTC']>= Start_date):
        return True
    else:
        return False
Travix_data['CancelledOrders_Phase1&2'] = Travix_data.apply (lambda row: Order_cancelled (row),axis=1)

In [59]:
Travix_data['CancelledOrders_Phase1&2'].value_counts()

False    485721
True      16858
Name: CancelledOrders_Phase1&2, dtype: int64

In [60]:
#Check
Travix_data[Travix_data['Phase']=='Phase1&2']['OrderCanceled'].value_counts()

False    485720
True      16858
Name: OrderCanceled, dtype: int64

In [61]:
#drop cancelled orders in Phase 1&2
#travix_base_df.drop(travix_base_df[travix_base_df['CancelledOrders_Phase1&2']==True].index, inplace=True)

In [62]:
#Check
Travix_data['CancelledOrders_Phase1&2'].value_counts()

False    485721
True      16858
Name: CancelledOrders_Phase1&2, dtype: int64

# Create a pivot table

In [63]:
#select columns for the pivot table
clmns_to_keep_df=Travix_data[['OrderDateUTC','Order_year','Order_month','Orderweek','Haul','Policy_type','PolicyName','Insurance_purch','Travix_booking','58%Insurance_Premium', 'Brand/Channel', 'Travix_revenue', 'MP_fee_exc_tax&HS', 'CancelledOrders_Phase1&2', 'TotalRetail_value', 'OneWayOrReturn']]

In [64]:
#segment data per orderdate, year, week, haul and policy type
travix_pivot_df=clmns_to_keep_df.pivot_table(index=['OrderDateUTC','Order_year','Order_month','Orderweek','CancelledOrders_Phase1&2','Haul', 'Brand/Channel','Policy_type','OneWayOrReturn'], aggfunc='sum')

In [65]:
travix_pivot_df.reset_index(inplace=True)

In [73]:
travix_pivot_df['Order_year'].value_counts()

2019    12085
2020     3843
Name: Order_year, dtype: int64

In [72]:
travix_pivot_df = travix_pivot_df[travix_pivot_df['Order_year']!='2018']

# Export data

In [74]:
#export to bi-report drive
travix_pivot_df.to_csv(export_BI_drive+'Travix YoY Data excluding duplicates_Split Combi products.csv')